In [1]:
import json
import requests

def print_neat(dictionary_for_viewing):
    print(json.dumps(dictionary_for_viewing, indent=4, sort_keys=True))
# TODO
# why would you update a root?
# How do you clone a scenario
# How do you create a whole new scenario based on a new branch
# Set up routines
# Set up studies
# Access results
# get all the scenarios where a specific agent branch is used


# It's easy to accidentally send a string with Python's requests
# request.post(..., json={'a': 1, 'b': 2}) works
# request.post(..., body=json.dumps({'a': 1, 'b': 2})) works
# request.post(..., json=json.dumps({'a': 1, 'b': 2})) sends a string instead of an object – this MR now explicitly blocks this
# request.post(..., body={'a': 1, 'b': 2}) sends binary data with the content-type application/x-www-form-urlencoded, which is blocked

### Helper Functions

In [2]:
with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']
    # API_KEY = json.load(file)['ASTAGE_API_KEY'] # For work in dev environment
    # API_KEY = json.load(file)['LOCAL_API_KEY'] # For work in dev environment

headers = {'X_API_KEY': API_KEY, 'Content-Type': 'application/json'}

In [32]:
URL_BASE = 'https://api.sedaro.com'
# URL_BASE = 'https://api.astage.sedaro.com'
# URL_BASE = 'http://localhost:80' # For work in dev environment

URL_Stems = {
    'WORKSPACES': URL_BASE + '/workspaces/',
    'REPOSITORIES': URL_BASE + '/models/repositories/',
    'BRANCHES': URL_BASE + '/models/branches/',
    'PROJECTS': URL_BASE + '/projects/'
}

In [4]:
def get_all(object_type_url):
    q = requests.get(object_type_url, headers=headers)
    print('API URL: ', object_type_url)
    return [entry for entry in q.json()]

In [5]:
def get_item_by_name(object_type_url, name):
    q = requests.get(object_type_url, headers=headers)
    print('API URL: ', object_type_url)
    results = [entry for entry in q.json() if entry['name'] == name]
    if len(results) > 0:
        return results[0]
    else:
        return None

In [6]:
def get_item_by_id(object_type_url, id):
    url = object_type_url + id
    q = requests.get(url, headers=headers)
    print('API URL: ', url)
    if type(q.json()) is dict:
        return q.json()
    else:
        return None

In [7]:
# Also create block
def update_item(object_type_url, id, payload):
    item_id = id
    if object_type_url == URL_Stems['BRANCHES']:
        item_id = id + '/template'
    url = object_type_url + item_id
    q = requests.patch(url, headers=headers, data=payload)
    print('API URL: ', url)
    return q.json()

In [246]:
# When you create a block, you are technically updating/patching the branch
# def create_block(object_type_url, id, payload):
#     return update_item(object_type_url, id, payload)

In [8]:
# repos need a special method to get them by name
def get_repo_by_name(workspace_id, repo_name):
    ws = get_item_by_id(URL_Stems['WORKSPACES'], workspace_id)
    list_of_repos = []
    for repository in ws['repositories']:
        if repository['name'] == repo_name:
            list_of_repos.append(repository)
    # Or using a Python list comprehension
    # list_of_repos = [repository for repository in workspace['repositories'] if repository['name'] == repo_name]
    if len(list_of_repos) > 0:
        return list_of_repos[0]
    else:
        return None

In [9]:
def get_branch_by_name(repo_id, branch_name):
    rp = get_item_by_id(URL_Stems['REPOSITORIES'], repo_id)
    list_of_branches = []
    for branch in rp['branches']:
        if branch['name'] == branch_name:
            list_of_branches.append(branch)
    # Or using a Python list comprehension
    # list_of_branches = [branch for branch in rp['branches'] if branch['name'] == branch_name]
    if len(list_of_branches) > 0:
        return list_of_branches[0]
    else:
        return None

In [10]:
def create_repo(object_type_url, payload):
    q = requests.post(object_type_url, headers=headers, data=payload)
    print('API URL: ', object_type_url)
    return q.json()

In [11]:
def create_branch(object_type_url, id, payload):
    url = object_type_url + id
    q = requests.post(url, headers=headers, data=payload)
    print('API URL: ', url)
    return q.json()

In [12]:
def get_block_instances(branch_id, block_type):
    branch = get_item_by_id(URL_Stems['BRANCHES'], branch_id)
    if 'error' not in branch:
        block_ids = branch['data']['index'][block_type]
        blocks = {}
        for block_id in block_ids:
            block = branch['data']['blocks'][block_id]
            blocks[block['id']]=block
        if len(block_ids) > 0:
            return {'UIDs': block_ids, 'blocks': blocks}
        else:
            return {'UIDs': None, 'blocks': None}
    else:
        return branch

In [13]:
def get_block_instance_by_name(branch_id, block_type, name):
    block_results = get_block_instances(branch_id, block_type)
    print(block_results)
    if 'error' not in block_results:
        blocks = []
        if block_results['UIDs'] is not None:
            for id in block_results['UIDs']:
                b = block_results['blocks'][id]
                if b['name'] == name:
                    blocks.append(b)
        if len(blocks) > 0:
            return blocks[0]
        else:
            return None
    else:
        return block_results

### Workspaces

In [22]:
workspace_from_name = get_item_by_name(URL_Stems['WORKSPACES'], "LC Demos")
workspace_id = workspace_from_name['id']
# Watchout, workspaces can have the same name
workspace_id

API URL:  https://api.sedaro.com/workspaces/


'PLbmFrM26h37tNj6XjVvQD'

In [ ]:
# Get workspace by id
workspace = get_item_by_id(URL_Stems['WORKSPACES'], workspace_id)
print_neat(workspace)

In [ ]:
# Rename a workspace
workspace_data = {'id': workspace_id, 'name': 'LC Demos'}
update_item(URL_Stems['WORKSPACES'], workspace_id, json.dumps(workspace_data))
# returns the workspace

In [ ]:
ws = get_item_by_id(URL_Stems['WORKSPACES'], 'PLbmFrM26h37tNj6XjVvQD')

In [ ]:
print_neat(ws)
# get_repo_by_name('PMDvpzbKWZNnxPssSnXSgk','')

### Projects

In [33]:
prj = get_item_by_id(URL_Stems['PROJECTS'], 'PLzRbHslBxmcnmRRntmSpD')

API URL:  https://api.sedaro.com/projects/PLzRbHslBxmcnmRRntmSpD


In [34]:
print_neat(prj)

{
    "createdBy": {
        "dateCreated": "2023-04-26T18:54:35.200Z",
        "dateModified": "2024-05-06T17:19:36.051Z",
        "email": "lael.cox@sedarotech.com",
        "externalIdentityProvider": null,
        "firstName": "Lael",
        "id": "NTyuhTQSoD0PhFWUTCRiF",
        "incomplete": false,
        "isFlagged": false,
        "isVerified": true,
        "lastName": "Cox",
        "organization": "Sedaro",
        "organizationCountryCode": "US",
        "roles": [
            "NTz2gzIRRXIC1vkE1PEtV",
            "PJskg3GfQFffDH2ZfspGRf",
            "NalsCPtTsegMUKS_J1XwV",
            "NXkVfgZSZXPbmqCFzfUGV",
            "PLbmFtHFDvd4r2p6xWL2W6",
            "PLcDcrxBtHtN8RlpKSWJfW",
            "PLSd3NhpgNY2KNVGXn7f5Q",
            "PMDvq3Sf24bm9fCH8npSyy",
            "PKCbxCmbl6ptnqfmSrHHSj",
            "PMHfkfBNcWKrwpMcfG8nxy"
        ],
        "userCountryCode": "US",
        "workspaces": [
            "PMHfkcJQMZFPx9Nd7HJhCv",
            "PLSd3LprQLXmJ4M99B8Rb

### Repos

In [ ]:
# Create a Repository
repo_data = {'name': 'API_SC7', 'description': 'Test3',
                   'metamodelType': 'Spacecraft', 'workspace': workspace['id']}
new_repo = create_repo(URL_Stems['REPOSITORIES'], json.dumps(repo_data))
new_repo
# Returns the new repo

In [ ]:
# To get all your repos across all workspaces you belong to (this can take a while)
all_repos = get_all(URL_Stems['REPOSITORIES'])
for r in all_repos:
    print('Workspace: ', r['workspace']['id'], 'Repo: ', r['name'])

In [ ]:
# Get a list of repos in workspace
repos = workspace['repositories']
print_neat(repos)

In [ ]:
# Getting a repo by name has its own function
repo = get_repo_by_name(workspace['id'],'Sensing Sat')
print_neat(repo)

In [ ]:
# Get a repo by id
repository_from_id = get_item_by_id(URL_Stems['REPOSITORIES'], repo['id'])
print_neat(repository_from_id)

In [ ]:
repo

### Branches

In [ ]:
# Get list of branches
repo['branches']

In [ ]:
# Get a branch by name
branch = get_branch_by_name(repo['id'],'main')
branch

In [ ]:
# Get a branch by id
branch_from_id = get_item_by_id(URL_Stems['BRANCHES'], 'PLzRcBmvgpPF5xfYCWjg9v')
print_neat(branch_from_id)

In [ ]:
# Create a branch
branch_data = {'name': 'API Branch 8', 'description': 'Test Branch3',
                   'branchId': branch['id']}
create_branch(URL_Stems['BRANCHES'], branch['id'], payload=json.dumps(branch_data))

### Working with Blocks

In [ ]:
# Pattern to create
url = URL_Stems['BRANCHES'] + id + '/template'

data = {'blocks': [{
    'id': '',  # optional
    'type': '',
    # 'property': 'value'
    }]}

q = requests.post(url, headers=headers, data=json.dumps(data))
print(json.dumps(q.json(), indent=2))

In [ ]:
get_block_instances('PLzRcBmvgpPF5xfYCWjg9v', 'BodyFrameVector')

In [ ]:
get_block_instance_by_name('PLzRcBmvgpPF5xfYCWjg9v', 'BodyFrameVector', '+X')

### Other Useful Blocks

In [ ]:
# To create a block, you are updating the branch
data = {'blocks': [{
            'name': 'VT6',
            'type': 'BodyFrameVector',
            'definitionType': 'SPHERICAL_ANGLES',
            'definitionParams': {
                'phi': 7,
                'theta': 2
            }
        }]}

update_item(URL_Stems['BRANCHES'], 'PM47vRx4XcgfXX8LgNzwDK', json.dumps(data))
# print(json.dumps(q.json(), indent=2))

In [ ]:
#Invoke ideal pointing mode, you are updating the branch
data = {'blocks': [{
    'type': "IdealOrbitalAttitudeDynamics"
}]}
update_item(URL_Stems['BRANCHES'], 'PM47vRx4XcgfXX8LgNzwDK', json.dumps(data))

# When you switch between pointing modes, it just overwrites the block
# These create and update actions return the whole branch